<a href="https://colab.research.google.com/github/IleniIsac/isac/blob/main/Variational_AutoEncode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Define sampling layer
Custom layer: subclassing the abstract layer class and defining the call method which describes how a tensor is transformed by the layer

Subclass inherits sttributes and methods of parent class

In [ ]:

import numpy as np
from tensorflow.keras import datasets
import matplotlib.pyplot as plt



# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()

# Normalize the images to the range [0, 1]
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
x_train.shape

(60000, 28, 28)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, metrics, backend as K
from tensorflow.keras.losses import binary_crossentropy

In [ ]:
class Sampling(layers.Layer):
  """Uses (z_mean, z_log_var) to sample z, the vector encoding an item of
  clothing."""
  def call(self, inputs):
    z_mean, z_log_var = inputs
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

# Defining the Encoder

In [ ]:
encoder_input = layers.Input(shape=(32, 32, 3),name="encoder_input")
x = layers.Conv2D(32, (3, 3), strides=2, activation="relu", padding="same")(encoder_input)
x = layers.Conv2D(64, (3, 3), strides=2, activation="relu", padding="same")(x)
x = layers.Conv2D(128, (3, 3), strides=2, activation="relu", padding="same")(x)
shape_before_flattening = K.int_shape(x)[1:]
x = layers.Flatten()(x)
x = layers.Dense(256, activation="relu")(x)
z_mean = layers.Dense(2, name="z_mean")(x) #mapping the flattened layer to z-mean, 2 is the dimnesion of the latent space
z_log_var = layers.Dense(2, name="z_log_var")(x) #mapping the flattened layer to z-mean, 2 is the dimnesion of the latent space
z = Sampling()([z_mean, z_log_var])
encoder = models.Model(encoder_input, [z_mean, z_log_var, z], name="encoder")

# Define loss function

In [ ]:
# Wrap the TensorFlow operations within a custom Keras layer
class KLLossLayer(layers.Layer):
    """Calculates the KL divergence loss."""
    def call(self, inputs):
        z_mean, z_log_var = inputs
        kl_loss = -0.5 * tf.reduce_sum(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=-1)
        return kl_loss

# Instantiate the custom layer and pass the KerasTensors
kl_loss = KLLossLayer()([z_mean, z_log_var])

#Define the decoder

In [ ]:
decoder_input = layers.Input(shape=(2,), name="decoder_input")
x = layers.Dense(np.prod(shape_before_flattening))(decoder_input)
x = layers.Reshape(shape_before_flattening)(x)
x = layers.Conv2DTranspose(128, (3, 3), strides=2, activation = 'relu',padding="same")(x)
x = layers.Conv2DTranspose(64, (3, 3), strides=2, activation = 'relu',padding="same")(x)
x = layers.Conv2DTranspose(32, (3, 3), strides=2, activation = 'relu',padding="same")(x)
decoder_output = layers.Conv2D(3, (3, 3), strides = 1, activation="sigmoid",padding="same", name="decoder_output")(x)
decoder = models.Model(decoder_input, decoder_output)
decoder.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ decoder_input (InputLayer)           │ (None, 2)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2048)                │           6,144 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 4, 4, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose (Conv2DTranspose)   │ (None, 8, 8, 128)           │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_1 (Conv2DTranspose) │ (None, 16, 16, 64)          │          73,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_2 (Conv2DTranspose) │ (None, 32, 32, 32)          │          18,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ decoder_output (Conv2D)              │ (None, 32, 32, 3)           │             867 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 246,851 (964.26 KB)

 Trainable params: 246,851 (964.26 KB)

 Non-trainable params: 0 (0.00 B)

# Train VAE

In [ ]:
x_train = tf.image.resize(x_train[..., tf.newaxis], [32, 32])
x_train = tf.repeat(x_train, 3, axis=-1)

vae = VAE(encoder, decoder)
vae.compile(optimizer="adam")
vae.fit(
    x_train,
    epochs=5,
    batch_size=100
)

NameError: name 'VAE' is not defined

#Making Predictions

In [ ]:
import tensorflow as tf

example_images = x_test[:5000]

# Print the shape of x_test to understand its structure
print("Shape of x_test:", example_images.shape)

if len(example_images.shape) == 4:
    # Drop the last dimension if it's unnecessary
    example_images = example_images[:,:,:,0]
    # Reshape to add the channel dimension
    example_images = example_images.reshape((-1, 28, 28, 1))
elif len(example_images.shape) == 3:
    # Reshape to add the channel dimension directly
    example_images = example_images.reshape((-1, 28, 28, 1))
else:
    print("Unexpected shape of x_test. Please check your data.")

example_images_resized = tf.image.resize(example_images, (32, 32)) # Resize images to (32, 32)

# If the images are grayscale and you need to convert them to RGB, you can
# duplicate the grayscale channel three times:
example_images_rgb = tf.repeat(example_images_resized, 3, axis=-1)

predictions = vae.predict(example_images_rgb) # Calculate predictions
plt.figure(figsize=(1,1))
plt.imshow(predictions[2][51])

In [ ]:
plt.figure(figsize=(0.7,0.7))
plt.imshow(example_images[51])



#Visualize sample image

In [ ]:
x_train[0]
plt.figure(figsize=(0.7,0.7))
plt.imshow(x_train[100])

In [ ]:
# Assuming 'example_images' is a NumPy array of grayscale images
example_images = example_images.reshape((-1, 28, 28, 1))  # Add channel dimension for grayscale
example_images_resized = tf.image.resize(example_images, (32, 32))  # Resize to match encoder input
example_images_rgb = tf.repeat(example_images_resized, 3, axis=-1)  # Convert to RGB by repeating channels

embeddings = encoder.predict(example_images_rgb)  # Now the input should match the encoder's expectation
embeddings[0].shape  # Check the shape of the embeddings

#Mapping the low dimensional latent space

In [ ]:
# embeddings = encoder.predict(example_images) #encodings to low dimensional space of example images
plt.figure(figsize=(8, 8))

# Calculate the appropriate first dimension for reshaping
num_points = embeddings[0].size // 2  # Total elements divided by the second dimension (2)

# Reshape embeddings to 2D for plotting
embeddings_2d = embeddings[0].reshape(num_points, 2)

# Adjust the number of color values to match the number of data points
# Assuming y_test corresponds to the labels for all data points in embeddings
color_values = y_test.flatten()[:num_points]  # Flatten y_test and slice to match the number of points

# Normalize color values to be between 0 and 1 for the colormap
normalized_color_values = (color_values - color_values.min()) / (color_values.max() - color_values.min())

# Use the reshaped embeddings for plotting
plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1],
            c=normalized_color_values, cmap='tab10', alpha=0.5, s=3)
plt.show()

In [ ]:
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
# embeddings = encoder.predict(example_images) #encodings to low dimensional space of example images
plt.figure(figsize=(8, 8))
plt.scatter(embeddings[0][:, 0], embeddings[0][:, 1],  c=color_values, cmap='tab10', alpha=0.5, s=3)

color_values=y_test[:5000]

plt.colorbar(ticks=range(10), label='Categories')


offset=[300,300]# distance where to display the actual
for index in [51,210]:
    x0, y0 = embeddings[0][index]# get the 2D embedding of the selected image
    image = example_images[index]  #get the actual image from mnist dataset

    # Create an OffsetImage
    imagebox = OffsetImage(image, zoom=1.5)  # Adjust zoom as needed

    # Create an AnnotationBbox with an offset
    ab = AnnotationBbox(
        imagebox,
        (x0, y0),
        frameon=False,
        xybox=offset,
        xycoords='data',
        boxcoords="offset points",
        pad=0.5,
        arrowprops=dict(arrowstyle="->", color='red')
    )

    # Add AnnotationBbox to the plot
    plt.gca().add_artist(ab)

# Adjust plot limits to ensure there's space for the annotations
# plt.xlim(min(embeddings[:, 0]) - 1, max(embeddings[:, 0]) + 1)
# plt.ylim(min(embeddings[:, 1]) - 1, max(embeddings[:, 1]) + 1)

# Show plot
plt.show()

In [ ]:
# image.shape
example_images=x_test[:5000]
plt.figure(figsize=(0.7,0.7))
plt.imshow(example_images[210])


#Generating new images

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Assuming you have the decoder model defined and trained
# Here, `latent_dim` is the dimensionality of the latent space. For example, 2.

latent_dim = 2  # This should match the dimensionality used in your VAE model

# Function to generate new images
def generate_new_images(decoder, num_images=10):
    # Sample random points from a standard normal distribution
    random_latent_vectors = np.random.normal(size=(num_images, latent_dim))

    # Decode these latent vectors to generate new images
    generated_images = decoder.predict(random_latent_vectors)

    # Plot the generated images
    plt.figure(figsize=(2,2))
    for i in range(num_images):
        ax = plt.subplot(1, num_images, i + 1)
        plt.imshow(generated_images[i].reshape(32, 32,3))  # Assuming image size is 32x32x3
        plt.axis('off')
    plt.show()

# Generate and display new images
generate_new_images(decoder, num_images=3)
